# Docker *containers* to the rescue

<img src="http://j.mp/1QUUrKp">

## Containers and Lightweigth virtualization

<img src="http://danstroot.com/assets/img/dockervsvm.png">


- Old concept
    * LXC
- Virtualization over Kernel specs
    * CGROUPS (users and disk)
    * NameSpaces (processes)
- Performance oriented
    * But limited to UNIX distros

## Docker engine

<img src='https://denibertovic.com/talks/supercharge-development-env-using-docker/img/what_is_docker.png'>


* Makes containers **easy**
* Written in GO
* Based on libcontainers
    - to be implemented for Windows too

## Docker (client) commands

In [ ]:
%%bash 
# Extra clean to show examples
docker rm -f $(docker ps -a -q)

How docker commands work:

```bash
# Generic client
docker DOCKER_COMMAND <options> 

# Run command
docker run <options> IMAGE BASH_COMMAND
```

Execute a container

In [ ]:
! docker run -it ubuntu:15.10 ls /

Launch a database server on the fly

In [ ]:
! docker run \
    --detach \
    --name mydb \
    -e POSTGRES_USER=paulie \
    -e POSTGRES_PASSWORD=mypass \
    -e POSTGRES_DB=justadb \
    postgres

You can freeze your containers!

In [ ]:
! docker stop mydb

In [ ]:
! docker start mydb

You may run commands on a running container

(e.g. to open a bash inside)

In [ ]:
! docker exec -it mydb ls /var/lib

## Deeper Docker

## Containers and processes

In [ ]:
! docker ps

In [ ]:
! docker ps -a

## Networking

The old way: linking containers to each other

In [ ]:
! docker run -it --link mydb:server -e PGPASSWORD=mypass postgres \
    psql \
    -h server \
    -U paulie \
    -d justadb \
    -c '\l' #list databases

In [ ]:
! docker run -it --link acontainerthatdoesnotexist:linked ubuntu:15.10 bash

The new and more powerful way: create and manage networks

In [ ]:
! docker network create --driver=bridge mynet

In [ ]:
! docker run -d -p 80:80 --net=mynet clue/adminer

Now go to [localhost](http://local.docker)

In [ ]:
# Add a linked database on a running container (adminer)
! docker run -d --name mydb --net=mynet \
    -e POSTGRES_USER=paulie \
    -e POSTGRES_PASSWORD=mypass \
    -e POSTGRES_DB=justadb \
    postgres

## Docker images

What containers do run

There is an official and public Docker Hub

(the GitHub of docker images)

In [ ]:
! docker search nginx

You can build your own image

In [ ]:
%%writefile Dockerfile
FROM alpine
RUN apk update && apk upgrade
RUN apk add python3

In [ ]:
%cat Dockerfile

Then use it

In [ ]:
! docker build -t mydockerimage .

In [ ]:
# Test the image you created
! docker run -it \
    mydockerimage \
    python3 -c "print(5*5)"

And push it to the public hub

In [ ]:
! docker push mydockerimage

- CINECA inside the Docker Hub:
https://hub.docker.com/u/cineca/
- My images:
https://hub.docker.com/u/pdonorio/

## Volumes and persistence

Containers are volatile

In [ ]:
! docker run --rm busybox touch /tmp/justatest

In [ ]:
! docker run -it busybox sh -c "ls /tmp/*"

If we launch a database server, and that containers dies... the data is gone.

Let's use the docker volumes

In [ ]:
! docker volume ls

In [ ]:
! docker run -it \
    -v persistence_test:/tmp \
    busybox touch /tmp/justatest

In [ ]:
! docker volume ls

In [ ]:
! docker run -it \
    -v persistence_test:/tmp \
    busybox sh -c "ls /tmp/*"

You may also:

* Mount a directory from the hosting machine as a volume
* Launch a data-only-container to share a volume across multiple containers
    - volume-from command

## Composing

How easy to make separate containers per app, and then compose them in your stack

A load balancer for web application with a proxy

In [ ]:
! docker run -d --name web1 -h test1 tutum/hello-world

In [ ]:
! docker run -d --name web2 -h test2 tutum/hello-world

In [ ]:
! docker run -d -p 80:80 --link web1:web1 --link web2:web2 tutum/haproxy

Go check it live now: [link](http://local.docker)

Meet the "compose" configuration

In [ ]:
%%writefile docker-compose.yml

web:
    image: tutum/hello-world
    
proxy:
    image: tutum/haproxy
    ports:
        - 80:80
    links:
        - web


In [ ]:
! docker-compose up -d

In [ ]:
! docker-compose scale web=3

In [ ]:
# Haproxy should be forced to restart, so it can see all the istances
! docker-compose up --force-recreate --no-deps -d proxy

In [ ]:
! docker-compose ps